In [1]:
import pandas as pd
import re
from keybert import KeyBERT

/Users/yumatakechi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [9]:
# fill null values with empty string to avoid error
cases_df = pd.read_csv('../cases.csv')
cases_df = cases_df.dropna(subset=['body']).fillna('')

In [10]:
print(len(cases_df))

2241


In [17]:
# remove URLs and special texts, and lower case
def clean_text(text):
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'[^A-Za-z0-9\s.,?]', '', text)
    text = text.lower()

    return text

In [19]:
# join the the relevant columns for each row to create a document
documents = []
for index, row in cases_df.iterrows():
    joined_text = ': '.join([(row['title']), (row['description']), (row['body'])])
    doc = clean_text(joined_text)
    
    documents.append(doc)

In [20]:
for doc in documents[:10]:
    print(doc)

british columbia citizens assembly on electoral reform an independent, nonpartisan assembly of 160 randomly selected bc residents, mandated to examine the current provincial election system and suggested alternatives. their final report and recommendation was submitted to the public and the final decision was put to a referendum.  problems and purpose  the  citizens assembly  on electoral reform was a body created by the government of british columbia, canada. the assembly was charged with investigating and recommending changes to improve the electoral system of the province. the body was composed of 160 citizens selected at random from throughout the province. these members met approximately every other weekend for one year to deliberate about alternative voting arrangements. after issuing their recommendations, the question would be put to the electorateatlarge in a  referendum  held concurrently with the 2005 provincial election.  background history and context  in 2004, 160 randoml

In [21]:
kw_model = KeyBERT()

In [22]:
# Defines a function that applies KeyBERT to each document
def extract_keywords(doc):
    return kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 2), use_maxsum=True, nr_candidates=20, stop_words='english', top_n=5)

In [24]:
# extract the top 5 relevant keywords (phrases with 1 to 3 words) from each document
keywords = [extract_keywords(doc) for doc in documents[:10]]
print(keywords[9])

[('alsoparticipatory budgetingreferences', 0.4435), ('budget issues', 0.4472), ('city daly', 0.4669), ('city participatory', 0.488), ('californian municipalities', 0.5307)]


In [65]:
len(keywords)

10

In [66]:
# separate the tuple into phrases and scores, and storing phrases as words by splitting 
keyword_list = []
score_list = []
for keyword in keywords:
    keyword_list.append(keyword[0])
    score_list.append(keyword[1])

In [67]:
cases_df['keywords'] = keyword_list
cases_df['scores'] = score_list

ValueError: Length of values (10) does not match length of index (2241)